In [1]:
from Database import Database
import sys
import multiprocessing as mp
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def get_project_ids(db):
    db.cursor.execute("""SELECT *
                            FROM project""")
    projects = db.cursor.fetchall()
    dict_project = dict()
    for proj in projects:
        dict_project[proj['ID']] = proj['project_ID']
    return dict_project

In [3]:
 path_db = 'D:/Hanze_Groningen/STAGE/db_laatste_copy.db' #'D:/Hanze_Groningen/STAGE/DATAB/copydatabase_C.db'
# Database connection
db = Database(path_db)
dict_project = get_project_ids(db)
print(len(dict_project))

43


In [4]:
df = pd.read_sql('''SELECT project.cancer, sum_dosage_GT.donor_project_ID, 
                            sum_dosage_GT.donor_ID, sum_dosage_GT.snp_ID, 
                            snp.chr, snp.pos_start, snp.pos_end 
                    FROM project, sum_dosage_GT, snp 
                    WHERE sum_dosage_GT.snp_ID=snp.ID AND 
                              sum_dosage_GT.donor_project_ID = project.ID AND 
                              (sum_dosage_GT.GT2 = 1 OR sum_dosage_GT.GT2 = 2) AND 
                              sum_dosage_GT.total_read_count_sum >= 33;''', db.mydb_connection)

In [5]:
print('HEAD')
print(df.head())
print(len(set(df['cancer'])))
print(set(df['cancer']))
print(len(set(df['donor_project_ID'])))
print(set(df['donor_project_ID']))
print(len(set(df['donor_ID'])))
print(len(set(df['snp_ID'])))
print(len(df))
print(set(df['chr']))

HEAD
          cancer  donor_project_ID  donor_ID  snp_ID chr  pos_start    pos_end
0  Head and neck                 1         1      41  11  101465614  101465614
1  Head and neck                 1         1     151  11  103127113  103127113
2  Head and neck                 1         1     197   8   82046917   82046917
3  Head and neck                 1         1     216  11  107789760  107789760
4  Head and neck                 1         1     241  14   48732779   48732779
15
{'Uterus', 'Bladder', 'Stomach', 'Brain', 'Colorectal', 'Bone', 'Blood', 'Gall Bladder', 'Cervix', 'Liver', 'Mesenchymal', 'Head and neck', 'Breast', 'Kidney', 'Ovary'}
40
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43}
2169
5651989
5855204
{'20', '14', '2', 'MT', '10', '17', '4', '11', '22', '7', '6', '1', '5', '19', '21', '12', 'X', '18', '15', '16', 'Y', '8', '13', '3', '9'}


In [6]:
df_fil = df.loc[df['chr'] != 'MT']
df_fil['chr'] ='chr' + df_fil['chr'].astype(str)

<ipython-input-6-5a2c1653c4d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['chr'] ='chr' + df_fil['chr'].astype(str)


In [7]:
breast_cancer = df_fil.loc[df_fil['cancer'] == 'Breast']
breast_cancer1 = breast_cancer[['chr', 'pos_start', 'pos_end']]
breast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)

<ipython-input-7-bb9243d632d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)


In [8]:
nonbreast_cancer = df_fil.loc[df_fil['cancer'] != 'Breast']
nonbreast_cancer1 = nonbreast_cancer[['chr', 'pos_start', 'pos_end']]
nonbreast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)

<ipython-input-8-4f38133a77e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonbreast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)


In [9]:
breast_cancer['snp'] = breast_cancer['chr'] + '_' + breast_cancer['pos_start'].map(str) + '_' + breast_cancer['pos_end'].map(str)
print(len(list(set(breast_cancer['donor_ID']))))
num_donor_breast = len(list(set(breast_cancer['donor_ID'])))
breast_cancer

285


<ipython-input-9-8adcad4a8fd6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breast_cancer['snp'] = breast_cancer['chr'] + '_' + breast_cancer['pos_start'].map(str) + '_' + breast_cancer['pos_end'].map(str)


,cancer,donor_project_ID,donor_ID,snp_ID,chr,pos_start,pos_end,snp
776755,Breast,25,1305,1979425,chr9,104350785,104350785,chr9_104350785_104350785
776756,Breast,25,1305,3656805,chr1,25646616,25646616,chr1_25646616_25646616
776757,Breast,25,1305,3656808,chr1,25646640,25646640,chr1_25646640_25646640
776758,Breast,25,1305,3656812,chr1,29155652,29155652,chr1_29155652_29155652
776759,Breast,25,1305,3656831,chr1,61862224,61862224,chr1_61862224_61862224
...,...,...,...,...,...,...,...,...
993973,Breast,28,2057,8740839,chr8,48761994,48761994,chr8_48761994_48761994
993974,Breast,28,2057,8740853,chr2,54494456,54494456,chr2_54494456_54494456
993975,Breast,28,2057,8740862,chr22,46712078,46712078,chr22_46712078_46712078
993976,Breast,28,2057,8740870,chr17,33772618,33772618,chr17_33772618_33772618


In [10]:
nonbreast_cancer['snp'] = nonbreast_cancer['chr'] + '_' + nonbreast_cancer['pos_start'].map(str) + '_' + nonbreast_cancer['pos_end'].map(str)
print(len(list(set(nonbreast_cancer['donor_ID']))))
num_donor_nonbreast = len(list(set(nonbreast_cancer['donor_ID'])))
nonbreast_cancer

<ipython-input-10-a350e5104874>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonbreast_cancer['snp'] = nonbreast_cancer['chr'] + '_' + nonbreast_cancer['pos_start'].map(str) + '_' + nonbreast_cancer['pos_end'].map(str)


1884


,cancer,donor_project_ID,donor_ID,snp_ID,chr,pos_start,pos_end,snp
0,Head and neck,1,1,41,chr11,101465614,101465614,chr11_101465614_101465614
1,Head and neck,1,1,151,chr11,103127113,103127113,chr11_103127113_103127113
2,Head and neck,1,1,197,chr8,82046917,82046917,chr8_82046917_82046917
3,Head and neck,1,1,216,chr11,107789760,107789760,chr11_107789760_107789760
4,Head and neck,1,1,241,chr14,48732779,48732779,chr14_48732779_48732779
...,...,...,...,...,...,...,...,...
5855199,Mesenchymal,43,2996,21812208,chr7,102949431,102949431,chr7_102949431_102949431
5855200,Mesenchymal,43,2996,21812224,chr11,121028837,121028837,chr11_121028837_121028837
5855201,Mesenchymal,43,2996,21812247,chr1,155658167,155658167,chr1_155658167_155658167
5855202,Mesenchymal,43,2996,21812248,chr17,46607840,46607840,chr17_46607840_46607840


In [11]:
breast_dict = dict(Counter(list(breast_cancer['snp'])))
nonbreast_dict = dict(Counter(list(nonbreast_cancer['snp'])))


In [12]:
breast_snp_count_df = pd.DataFrame([breast_dict.keys(), breast_dict.values()]).T
breast_snp_count_df.columns= ['snp', 'counts']

max_breast = breast_snp_count_df['counts'].max()
breast_snp_count_df['counts'].max()

22

In [13]:
breast_snp_count_df

,snp,counts
0,chr9_104350785_104350785,1
1,chr1_25646616_25646616,1
2,chr1_25646640_25646640,1
3,chr1_29155652_29155652,1
4,chr1_61862224_61862224,1
...,...,...
217123,chr8_48761994_48761994,1
217124,chr2_54494456_54494456,1
217125,chr22_46712078_46712078,1
217126,chr17_33772618_33772618,1


In [14]:
nonbreast_snp_count_df = pd.DataFrame([nonbreast_dict.keys(), nonbreast_dict.values()]).T
nonbreast_snp_count_df.columns= ['snp', 'counts']

max_nonbreast = nonbreast_snp_count_df['counts'].max()
nonbreast_snp_count_df['counts'].max()

39

In [15]:
nonbreast_snp_count_df

,snp,counts
0,chr11_101465614_101465614,1
1,chr11_103127113_103127113,1
2,chr8_82046917_82046917,1
3,chr11_107789760_107789760,1
4,chr14_48732779_48732779,1
...,...,...
4677613,chr7_102949431_102949431,1
4677614,chr11_121028837_121028837,1
4677615,chr1_155658167_155658167,1
4677616,chr17_46607840_46607840,1


In [16]:
breast_snp_count_df[['chr', 'pos_start', 'pos_end']] = breast_snp_count_df['snp'].str.split('_', expand=True)
nonbreast_snp_count_df[['chr', 'pos_start', 'pos_end']] = nonbreast_snp_count_df['snp'].str.split('_', expand=True)

In [17]:
breast_snp_count_df

,snp,counts,chr,pos_start,pos_end
0,chr9_104350785_104350785,1,chr9,104350785,104350785
1,chr1_25646616_25646616,1,chr1,25646616,25646616
2,chr1_25646640_25646640,1,chr1,25646640,25646640
3,chr1_29155652_29155652,1,chr1,29155652,29155652
4,chr1_61862224_61862224,1,chr1,61862224,61862224
...,...,...,...,...,...
217123,chr8_48761994_48761994,1,chr8,48761994,48761994
217124,chr2_54494456_54494456,1,chr2,54494456,54494456
217125,chr22_46712078_46712078,1,chr22,46712078,46712078
217126,chr17_33772618_33772618,1,chr17,33772618,33772618


In [18]:
nonbreast_snp_count_df

,snp,counts,chr,pos_start,pos_end
0,chr11_101465614_101465614,1,chr11,101465614,101465614
1,chr11_103127113_103127113,1,chr11,103127113,103127113
2,chr8_82046917_82046917,1,chr8,82046917,82046917
3,chr11_107789760_107789760,1,chr11,107789760,107789760
4,chr14_48732779_48732779,1,chr14,48732779,48732779
...,...,...,...,...,...
4677613,chr7_102949431_102949431,1,chr7,102949431,102949431
4677614,chr11_121028837_121028837,1,chr11,121028837,121028837
4677615,chr1_155658167_155658167,1,chr1,155658167,155658167
4677616,chr17_46607840_46607840,1,chr17,46607840,46607840


In [19]:
breast_snp_count_df.drop('snp', axis=1, inplace=True)
nonbreast_snp_count_df.drop('snp', axis=1, inplace=True)

In [20]:
sort_breast_snp_count_df = breast_snp_count_df.sort_values('counts', ascending=False)
sort_nonbreast_snp_count_df = nonbreast_snp_count_df.sort_values('counts', ascending=False)

In [21]:
sort_breast_snp_count_df.rename(columns={'counts': 'counts_breast'}, inplace=True)
sort_nonbreast_snp_count_df.rename(columns={'counts': 'counts_nonbreast'}, inplace=True)

In [22]:
sort_breast_snp_count_df

,counts_breast,chr,pos_start,pos_end
522,22,chr3,178952085,178952085
6952,6,chr3,178936091,178936091
43780,4,chr14,105246551,105246551
12552,3,chr10,8111433,8111434
85707,2,chr21,31172543,31172543
...,...,...,...,...
72393,1,chr6,101793167,101793167
72394,1,chr3,23319452,23319452
72395,1,chr5,24034565,24034565
72396,1,chr3,196110911,196110911


In [23]:
sort_nonbreast_snp_count_df

,counts_nonbreast,chr,pos_start,pos_end
787050,39,chr16,23961453,23961453
771541,32,chr4,119972341,119972341
6720,28,chr12,25398284,25398284
781492,28,chr5,124474355,124474355
817636,26,chr4,14673408,14673408
...,...,...,...,...
1629711,1,chr21,30025421,30025421
1629712,1,chr22,23674453,23674453
1629713,1,chr20,46011295,46011295
1629714,1,chr2,4199709,4199709


In [24]:
sort_breast_snp_count_df.to_csv(f"D:/Hanze_Groningen/STAGE/R/breast_snp_count_df_maxcount_{max_breast}_donor_{num_donor_breast}_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)
sort_nonbreast_snp_count_df.to_csv(f"D:/Hanze_Groningen/STAGE/R/nonbreast_snp_count_df_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [25]:
# merge_b_nb = pd.concat([sort_breast_snp_count_df, sort_nonbreast_snp_count_df], ignore_index=True, sort=False)

#https://stackoverflow.com/questions/53645882/pandas-merging-101
merge_b_nb = sort_breast_snp_count_df.merge(sort_nonbreast_snp_count_df, on=['chr', 'pos_start', 'pos_end'], how='outer')

In [26]:
merge_b_nb[(merge_b_nb['counts_breast']>=1) & (merge_b_nb['counts_nonbreast']>=1)]

,counts_breast,chr,pos_start,pos_end,counts_nonbreast
0,22,chr3,178952085,178952085,7
1,6,chr3,178936091,178936091,6
2,4,chr14,105246551,105246551,3
7,2,chr3,178936082,178936082,2
9,2,chr2,30161804,30161804,1
...,...,...,...,...,...
215577,1,chr11,129489147,129489147,1
215655,1,chr4,42282418,42282418,1
215965,1,chr4,25065454,25065454,1
215985,1,chr3,27961891,27961891,1


In [27]:
merge_b_nb

,counts_breast,chr,pos_start,pos_end,counts_nonbreast
0,22,chr3,178952085,178952085,7
1,6,chr3,178936091,178936091,6
2,4,chr14,105246551,105246551,3
3,3,chr10,8111433,8111434,NaN
4,2,chr21,31172543,31172543,NaN
...,...,...,...,...,...
4893919,NaN,chr21,30025421,30025421,1
4893920,NaN,chr22,23674453,23674453,1
4893921,NaN,chr20,46011295,46011295,1
4893922,NaN,chr2,4199709,4199709,1


In [28]:
merge_b_nb.to_csv(f"D:/Hanze_Groningen/STAGE/R/ALL_nan_df_BREAST_maxcount_{max_breast}_donor_{num_donor_breast}_NONBREAST_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_BOTH_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [29]:
merge_b_nb_0 = merge_b_nb.fillna(0)
merge_b_nb_0.to_csv(f"D:/Hanze_Groningen/STAGE/R/ALL_0_df_BREAST_maxcount_{max_breast}_donor_{num_donor_breast}_NONBREAST_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_BOTH_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [30]:
merge_b_nb_0

,counts_breast,chr,pos_start,pos_end,counts_nonbreast
0,22,chr3,178952085,178952085,7
1,6,chr3,178936091,178936091,6
2,4,chr14,105246551,105246551,3
3,3,chr10,8111433,8111434,0
4,2,chr21,31172543,31172543,0
...,...,...,...,...,...
4893919,0,chr21,30025421,30025421,1
4893920,0,chr22,23674453,23674453,1
4893921,0,chr20,46011295,46011295,1
4893922,0,chr2,4199709,4199709,1
